# Проект машинное обучение для текстов с использованием BERT

Дмитрий, привет! Прошу прощения, что удалил комментарии, видимо забыл вставить в новый ноутбук. Спасибо за твое ревью, высылаю новую версию проекта

In [1]:
!pip install -q  transformers catboost

     |████████████████████████████████| 5.5 MB 4.9 MB/s 
     |████████████████████████████████| 76.6 MB 1.3 MB/s 
     |████████████████████████████████| 7.6 MB 47.8 MB/s 
     |████████████████████████████████| 163 kB 65.1 MB/s 


In [34]:
import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_csv('toxic_comments.csv')
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
data.shape

(159292, 3)

Инициализируем BERT

In [7]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, do_basic_tokenize=True)

tokenized = data['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [22]:
model = transformers.BertModel.from_pretrained("unitary/toxic-bert").to('cuda')

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Следующая ячейка выолняется 1,5 часа

In [23]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings = model(batch.cuda(), attention_mask=attention_mask_batch.cuda())

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/1592 [00:00<?, ?it/s]

Собираем эмбеддинги и делим на тренировочную и тестовую выборки

In [24]:
features = np.concatenate(embeddings)
target = data['toxic']

In [29]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target[:159200],test_size=0.2)
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)

(127360, 768)
(31840, 768)
(127360,)
(31840,)


В качестве модели использую классификатор CatBoost, так как его можно обучать на GPU, что значительно упростит задачу

In [30]:
catboost = CatBoostClassifier(eval_metric='F1', task_type = 'GPU')

parameters = {'iterations': [100, 250, 500],
              'depth': [1, 2, 3, 4],
              'l2_leaf_reg': [0.2, 0.5, 1],
              'learning_rate': [0.001, 0.1, 0.5]
              }

Использую GridSearch для поиска оптимальных гиперпараметров

In [31]:
grid_search_result = catboost.grid_search(parameters,
                                          X=features_train,
                                          y=target_train,
                                          plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Выходные данные были обрезаны до нескольких последних строк (5000).
34:	learn: 0.9491313	test: 0.9407436	best: 0.9407436 (34)	total: 1.64s	remaining: 10.1s
35:	learn: 0.9494201	test: 0.9407895	best: 0.9407895 (35)	total: 1.69s	remaining: 10.1s
36:	learn: 0.9491460	test: 0.9406075	best: 0.9407895 (35)	total: 1.75s	remaining: 10s
37:	learn: 0.9497067	test: 0.9410172	best: 0.9410172 (37)	total: 1.79s	remaining: 9.98s
38:	learn: 0.9498077	test: 0.9408581	best: 0.9410172 (37)	total: 1.83s	remaining: 9.93s
39:	learn: 0.9496562	test: 0.9410628	best: 0.9410628 (39)	total: 1.89s	remaining: 9.91s
40:	learn: 0.9497524	test: 0.9408581	best: 0.9410628 (39)	total: 1.93s	remaining: 9.86s
41:	learn: 0.9499543	test: 0.9406763	best: 0.9410628 (39)	total: 1.98s	remaining: 9.8s
42:	learn: 0.9499087	test: 0.9406534	best: 0.9410628 (39)	total: 2.02s	remaining: 9.74s
43:	learn: 0.9500000	test: 0.9408353	best: 0.9410628 (39)	total: 2.07s	remaining: 9.68s
44:	learn: 0.9499952	test: 0.9404716	best: 0.9410628 (3

Training on fold [0/3]
0:	learn: 0.9385443	test: 0.9377179	best: 0.9377179 (0)	total: 130ms	remaining: 32.4s
1:	learn: 0.9406181	test: 0.9430365	best: 0.9430365 (1)	total: 250ms	remaining: 31s
2:	learn: 0.9423121	test: 0.9436456	best: 0.9436456 (2)	total: 381ms	remaining: 31.3s
3:	learn: 0.9418093	test: 0.9418875	best: 0.9436456 (2)	total: 487ms	remaining: 30s
4:	learn: 0.9420667	test: 0.9440389	best: 0.9440389 (4)	total: 627ms	remaining: 30.7s
5:	learn: 0.9429266	test: 0.9442577	best: 0.9442577 (5)	total: 772ms	remaining: 31.4s
6:	learn: 0.9429234	test: 0.9451800	best: 0.9451800 (6)	total: 906ms	remaining: 31.4s
7:	learn: 0.9439024	test: 0.9449424	best: 0.9451800 (6)	total: 1.03s	remaining: 31.3s
8:	learn: 0.9441995	test: 0.9451928	best: 0.9451928 (8)	total: 1.17s	remaining: 31.2s
9:	learn: 0.9447952	test: 0.9454123	best: 0.9454123 (9)	total: 1.31s	remaining: 31.4s
10:	learn: 0.9448080	test: 0.9444961	best: 0.9454123 (9)	total: 1.44s	remaining: 31.4s
11:	learn: 0.9450728	test: 0.94477

Обучим модель с лучшими параметрами параметрами

In [32]:
catboost2 = CatBoostClassifier(**grid_search_result.get('params'), task_type = 'GPU', eval_metric='F1')
catboost2.fit(features_train, target_train)

0:	learn: 0.9354901	total: 18.6ms	remaining: 4.64s
1:	learn: 0.9400701	total: 37.3ms	remaining: 4.62s
2:	learn: 0.9407682	total: 55ms	remaining: 4.53s
3:	learn: 0.9414315	total: 72.5ms	remaining: 4.46s
4:	learn: 0.9409941	total: 90.7ms	remaining: 4.44s
5:	learn: 0.9419275	total: 110ms	remaining: 4.46s
6:	learn: 0.9425501	total: 128ms	remaining: 4.45s
7:	learn: 0.9434401	total: 147ms	remaining: 4.44s
8:	learn: 0.9437696	total: 166ms	remaining: 4.43s
9:	learn: 0.9439615	total: 182ms	remaining: 4.36s
10:	learn: 0.9446426	total: 195ms	remaining: 4.24s
11:	learn: 0.9448837	total: 209ms	remaining: 4.14s
12:	learn: 0.9451503	total: 228ms	remaining: 4.15s
13:	learn: 0.9451816	total: 241ms	remaining: 4.06s
14:	learn: 0.9457190	total: 255ms	remaining: 4s
15:	learn: 0.9459439	total: 268ms	remaining: 3.92s
16:	learn: 0.9460337	total: 282ms	remaining: 3.87s
17:	learn: 0.9460421	total: 295ms	remaining: 3.8s
18:	learn: 0.9461318	total: 308ms	remaining: 3.75s
19:	learn: 0.9462831	total: 321ms	remainin

Оценим модель на тестовой выборке

In [33]:
predicted = catboost2.predict(features_test)
print(f1_score(target_test, predicted))
print(catboost2.get_best_score())

0.9440396837699582
{'learn': {'Logloss': 0.024001344843725462, 'F1': 0.9583445724634888}}


In [35]:
grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
logreg=LogisticRegression()
logreg_cv = RandomizedSearchCV(logreg,grid,cv=10)
logreg_cv.fit(features_train,target_train)

print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


tuned hpyerparameters :(best parameters)  {'penalty': 'l2', 'C': 0.1}
accuracy : 0.9893765703517587


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [36]:
logreg2 = LogisticRegression(**logreg_cv.best_params_)
logreg2.fit(features_train, target_train)
predictions = logreg2.predict(features_test)
print('F! score - ', f1_score(target_test, predictions))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


F! score -  0.948053961854551


# Вывод
Исходя из полученных метрик качества моделей, лучшая модель на GridSearchCV - Catboost. Необходимые метрики достигнуты, модель, обученная через GridSearchCV, предсказывает с необходимой метрикой: F1 > 0.75.